# 💼 Multi-Agent Sales Email Generator using OpenAI Agentic Framework

This project uses the **OpenAI Agents SDK** to build a multi-agent, tool-using system that generates, formats, and sends sales emails.

### 🔍 What This Project Showcases:
- **Multi-Agent Architecture**: Agents for writing in different tones (professional, witty, concise)
- **Tool Wrapping**: Agents are exposed as tools to higher-level agents
- **Structured Output**: Emails are returned in a validated `EmailOutput` format
- **Guardrails**: Input and output are validated with custom guardrail agents
- **Email Delivery**: Messages are sent via the SendGrid API
- **Gradio UI**: A simple frontend lets users input instructions and a recipient, then view the generated result

### 🛠 Technologies Used:
- OpenAI SDK + Agents
- Gradio for UI
- SendGrid for email sending
- Python + Pydantic for structure enforcement
- Google Colab for development


### 🛠️ Install Required Packages for Agentic Email App


In [ ]:
!pip install \
  openai==1.91.0 \
  sendgrid==6.12.4 \
  gradio==5.34.2 \
  pydantic==2.11.7 \
  openai-agents==0.0.19

### 📦 Import Core Libraries and Agentic Framework Components


In [ ]:
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail,output_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
from google.colab import userdata
import asyncio
import gradio as gr

### 🔐 Load and Verify API Keys from Colab Secrets

To use models and services in this project, you need API keys from the following providers. Once you register and obtain them, add them securely to your Colab session using the `colab_secrets`.

- 🔑 **OpenAI**: [platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)
- 🔑 **Google (Gemini)**: [console.cloud.google.com](https://console.cloud.google.com) → Enable **Generative Language API**
- 🔑 **DeepSeek**: [deepseek.com](https://deepseek.com) → Developer Portal
- 🔑 **Groq**: [console.groq.com](https://console.groq.com) → Create API Key under your account
- 🔑 **Anthropic**: [console.anthropic.com](https://console.anthropic.com) → API Keys tab
- ✉️ **SendGrid**: [app.sendgrid.com/settings/api_keys](https://app.sendgrid.com/settings/api_keys) → Create "Full Access" key to send emails

Paste each key into the Colab "Secrets" UI to access them securely.


In [ ]:
openai_api_key = os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
google_api_key = userdata.get("GOOGLE_API_KEY")
deepseek_api_key = userdata.get("DEEPSEEK_API_KEY")
groq_api_key = userdata.get("GROQ_API_KEY")
anthropic_api_key = userdata.get("ANTHROPIC_API_KEY")
sendgrid_api_key = userdata.get("SENDGRID_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if sendgrid_api_key:
    print(f"Sendgrid API Key exists")
else:
    print("Sendgrid API Key not set")

### 🤖 Agent Instructions for Sales Email Generation (Professional, Witty, and Concise Styles)


In [ ]:
instructions1 = """
You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
You write professional, serious cold emails.

If not explicitly provided, assume:
- Sender name is "Alice"
- Signoff is "Best regards"
- The company is "ComplAI"
- The recipient is a CEO
"""

instructions2 = """You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response.

Your output should be a structured object with the following fields:
- body: the main email content (addressed to 'Dear CEO')
- signoff: the signoff line (e.g., 'Best regards')
- sender_name: the name of the person sending the email (e.g., 'Alice')
Return only this structured output.

If not explicitly provided, assume:
- Sender name is "Alice"
- Signoff is "Best regards"
- The company is "ComplAI"
- The recipient is a CEO
"""

instructions3 = """You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails.

If not explicitly provided, assume:
- Sender name is "Alice"
- Signoff is "Best regards"
- The company is "ComplAI"
- The recipient is a CEO"""

### 🔌 Configure External LLM Clients and Models (Gemini, DeepSeek, Groq)

These base URLs and model clients allow you to connect with OpenAI-compatible LLM endpoints from third-party providers using the OpenAI Agents SDK. Each client is initialized with its respective API key and wrapped into an `OpenAIChatCompletionsModel` for agent use.


In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

### 📤 Define Structured Output Schema for Sales Emails


In [ ]:
class EmailOutput(BaseModel):
    body: str
    signoff: str
    sender_name: str

### 🤖 Define Sales Agents with Distinct Personalities and Models


In [ ]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model, output_type=EmailOutput)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

### 🛠️ Convert Sales Agents into Callable Tools for Email Generation


In [ ]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

### 📧 Define SendGrid Tool to Send HTML Emails

This tool wraps the SendGrid API for sending emails with a subject, recipient, and HTML-formatted body.  
**🔐 Note:** Replace `"you@example.com"` with your **verified sender email** in your SendGrid account to enable successful delivery.


In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str, recipient_email: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("you@example.com") # change it to your email
    to_email = To(recipient_email)
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### 🧠 Define Utility Agents for Subject Generation and HTML Conversion


In [ ]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

### 🧰 Group Email-Related Tools for Use by the Email Manager Agent


In [ ]:
email_tools = [subject_tool, html_tool, send_html_email]

### 📬 Define the Email Manager Agent to Format, Convert, and Send Emails

This agent orchestrates a multi-step email pipeline: generating a subject, converting the body to HTML, and sending the final email using available tools. It serves as the final handoff agent in the sales email workflow.


In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

### 🔗 Register Available Tools and Handoff Agents for the Sales Manager


In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

### 🧑‍💼 Define the Sales Manager Agent to Orchestrate Multi-Agent Email Generation

This agent acts as the decision-maker in the workflow. It evaluates responses from all three sales agents, selects the most effective email, and hands it off to the Email Manager for formatting and delivery. It never writes emails directly, ensuring tool-based generation only.


In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

### 🛡️ Define Input Guardrail to Detect Profanity in User Messages

This guardrail uses a dedicated agent to detect offensive language in user input. If profanity is found, it triggers a tripwire and prevents the agent from continuing. The check returns a structured output containing a flag and the first offending word.


In [ ]:
class ProfanityCheckOutput(BaseModel):
    contains_profanity: bool
    offending_word: str = ""

profanity_agent = Agent(
    name="Profanity Checker",
    instructions="Check if the user input contains profanity or offensive language. Return the first offending word if found.",
    output_type=ProfanityCheckOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def guardrail_profanity(ctx, agent, message):
    result = await Runner.run(profanity_agent, message, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info={"profanity_check": result.final_output},
        tripwire_triggered=result.final_output.contains_profanity
    )


### 🎯 Define Output Guardrail to Enforce Tone Consistency in Emails

This output guardrail uses a dedicated agent to validate whether the generated email aligns with the intended tone—professional, witty, or concise. If the tone is incorrect, it triggers a tripwire and returns feedback explaining the mismatch.


In [ ]:
# Define output structure
class ToneCheckOutput(BaseModel):
    tone_matched: bool
    feedback: str

# Define the guardrail agent
tone_checker_agent = Agent(
    name="Tone Checker",
    instructions="Check if the output email matches the intended tone (professional, concise, or witty). "
                 "If it doesn't, set tone_matched to false and explain why in the feedback.",
    output_type=ToneCheckOutput,
    model="gpt-4o-mini"
)

# Define the output guardrail function
@output_guardrail
async def check_email_tone(ctx, agent, output):
    result = await Runner.run(tone_checker_agent, output.response, context=ctx.context)
    return GuardrailFunctionOutput(
        output_info={"tone_check": result.final_output},
        tripwire_triggered=not result.final_output.tone_matched
    )

### 🧑‍💼🛡️ Define a Guarded Sales Manager Agent with Input and Output Guardrails

This version of the Sales Manager agent includes guardrails for both input and output:
- **Input Guardrail**: Detects profanity in user messages.
- **Output Guardrail**: Ensures the generated email matches the intended tone.

It maintains the same tool orchestration logic but adds safety and quality control to the generation process.


In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_profanity],
    output_guardrails=[check_email_tone]
    )

### 📨 Define Async Email Generation Function with Tracing

This function runs the guarded Sales Manager agent on a given message and returns the final output. It also uses `trace()` to enable OpenAI trace logging for debugging and analysis of the agent workflow.


In [ ]:
async def generate_email(message: str) -> str:
    with trace("Gradio SDR"):
        result = await Runner.run(careful_sales_manager, message)
    return result.final_output  # or formatted string if structured output

### 🔁 Define Synchronous Wrapper to Run Email Agent from Gradio UI

This function combines the user's message and recipient email into a single prompt, then synchronously runs the async agent pipeline using `asyncio.run()` so it can be used in the Gradio interface.


In [ ]:
def run_email_agent_sync(message: str, recipient_email: str) -> str:

    # Combine message and recipient into a single structured prompt
    full_message = f"{message}\n\nSend this email to: {recipient_email}"

    # Call the async function and return result
    return asyncio.run(generate_email(full_message))

### 🖥️ Build Gradio Interface for Multi-Agent Email Generation and Delivery

This interactive UI allows users to provide email instructions and a recipient address. It triggers the full OpenAI Agents workflow and displays the result. The layout is built with `gr.Blocks()` for a clean, responsive structure.


In [ ]:
with gr.Blocks(title="AI Sales Agent - OpenAI SDK Demo") as demo:
    gr.HTML("""
        <h1 style='text-align: center; font-size: 2.5em; font-weight: bold;'>
            💼 Multi-Agent Sales Email Generator
        </h1>
    """)

    with gr.Row():
        message_input = gr.Textbox(
            label="📝 Email Instructions",
            placeholder="e.g. Send a concise cold email to Dear CTO from Alice",
            lines=4
        )
        recipient_input = gr.Textbox(
            label="✉️ Recipient Email",
            placeholder="e.g. ceo@example.com"
        )

    generate_btn = gr.Button("🚀 Generate and Send Email")

    output_box = gr.Textbox(
        label="✅ Result",
        lines=12,
        interactive=False
    )

    generate_btn.click(fn=run_email_agent_sync, inputs=[message_input, recipient_input], outputs=output_box)

demo.launch(debug=True)


## Check out the trace:

https://platform.openai.com/traces